## eBay Baseball Card Data Scraper

In [369]:
import requests
import numpy as np
import pandas as pd
from scipy import stats
from bs4 import BeautifulSoup as bs

In [370]:
#link = "https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1312&_nkw=lewis+hamilton+chrome+auto&_sacat=0"

# link = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2019+panini+prizm+zion+williamson+248+psa+10&_pgn=3'
# mode = 'scrape'

In [371]:
# global variables 

keywords = '2019+panini+prizm+zion+williamson+248+psa+10'
base_link = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw='
end_link = '&_pgn='
link = base_link + keywords + end_link

names = []
prices = []
shippings = []
links = []
listing_types = []
grades = []



In [372]:
## functions

def scrape_listings(link):

    full_listings = []
    
    for i in range(1, 7): # needs to be edited statically, will parse from page eventually....

        page = requests.get(link + str(i))
        content = bs(page.content)
        soup = bs(page.content, 'html.parser')
        listings = soup.find_all('li', attrs={'class': 's-item'})
        full_listings.extend(listings)

    return full_listings

def find_name(listing):
    for name in listing.find_all('h3', attrs={'class': "s-item__title"}):
        if (str(name.find(text=True, recursive=False))!="None"):
                return str(name.find(text=True, recursive=False))
        else:
            return 'NA'

def find_grade(title):

    index = ''
    string = ''

    grading_companies = ['PSA', 'psa', 'BGS', 'bgs', 'HGA', 'hga', 'SGC', 'sgc']
    for company in grading_companies:
        index = title.find(company)
        if (index != -1):
            return title[index:index + 7]
    
    return 'NA'  

def find_price(listing):
    price = listing.find('span', attrs={'class':"s-item__price"}).get_text(strip=True)
    price = int(price.replace(',', '')[1:-3])
    return price

def find_ship(listing):
    cost = listing.find('span', attrs={'class':"s-item__shipping", 'class':"s-item__logisticsCost"})
    if cost is not None:
        return str(cost.find(text=True, recursive=False))
    else:
        return 'NA'

def find_type(listing):
    obo = listing.find('span', attrs={'class': 's-item__purchase-options-with-icon'})
    if (obo is not None):
        return 'BIN/OBO'
    else:
        return 'Auction'

def get_info(listing):

    prod_link = listing.select_one('a')['href']
    prod_name = find_name(listing)
    prod_grade = find_grade(prod_name)
    prod_price = find_price(listing)
    prod_ship_cost = find_ship(listing)
    prod_list_type = find_type(listing)

    def append_all():
        links.append(prod_link)
        names.append(prod_name)
        grades.append(prod_grade)
        prices.append(prod_price)
        shippings.append(prod_ship_cost)
        listing_types.append(prod_list_type)
    
    append_all()

In [373]:
listings = scrape_listings(link)

for listing in listings:
    get_info(listing)
    

In [379]:
zion = pd.DataFrame({
    "Titie": names,
    "Grade": grades,
    "Price": prices,
    "Listing Type": listing_types,
    "Shipping Cost": shippings,
    "Link": links})

In [380]:
zion

,Titie,Grade,Price,Listing Type,Shipping Cost,Link
0,Shop on eBay,NA,20,Auction,NA,https://ebay.com/itm/123456?hash=item28caef0a3...
1,ZION WILLIAMSON PRIZM ROOKIE CARD JERSEY #1 DU...,NA,49,BIN/OBO,Free shipping,https://www.ebay.com/itm/383332585170?hash=ite...
2,2019-20 Panini Prizm Silver MBA Zion Will...,PSA 10,999,BIN/OBO,+$9.99 shipping,https://www.ebay.com/itm/314079956157?epid=240...
3,2019 Panini Prizm Basketball Zion Williamson R...,PSA 10,142,Auction,+$6.00 shipping,https://www.ebay.com/itm/204052964071?hash=ite...
4,2019-20 Panini Prizm #248 Zion Williamson (RC)...,PSA 10!,0,Auction,+$4.70 shipping,https://www.ebay.com/itm/225099935011?epid=240...
...,...,...,...,...,...,...
276,2019 Panini Prizm #248 Zion Williamson Pelican...,PSA 10,994,BIN/OBO,+$5.50 shipping,https://www.ebay.com/itm/383873458015?epid=305...
277,2019-20 Panini Prizm Zion Williamson #248 Base...,PSA 10,500,Auction,+$4.50 shipping,https://www.ebay.com/itm/234639701719?hash=ite...
278,Zion Williamson 2019-20 Panini Prizm Rookie RC...,PSA 10,949,BIN/OBO,+$10.00 shipping,https://www.ebay.com/itm/383871199964?epid=240...
279,🔥2019-20 Panini Prizm Zion Williamson #248 PSA...,PSA 10,2000,BIN/OBO,+$5.00 shipping,https://www.ebay.com/itm/143960808582?epid=240...
